In [0]:
# ✅ Importar librerías necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType, StructType, StructField, BooleanType, IntegerType
import re

# ✅ Crear DataFrame de ejemplo
data = [
    (1, "  maria    jose  perez  "),
    (2, "CARLOS ramirez"),
    (3, "ana"),
    (4, "jUAN  ignacio LOPEZ"),
    (5, "luisa   fernanda   gomez  "),
]
df = spark.createDataFrame(data, ["id", "nombre_completo"])

df.show(truncate=False)

# ✅ UDF Estandariza los nombres y valida si tiene segundo nombre
def procesar_nombre(nombre):
    if not nombre:
        return None
    nombre_limpio = " ".join(nombre.strip().lower().split())  # Quitar espacios extras y poner en minúscula
    nombre_capitalizado = " ".join([w.capitalize() for w in nombre_limpio.split()])
    contiene_segundo_nombre = len(nombre_capitalizado.split()) > 2
    return f"{nombre_capitalizado} | Segundo nombre: {'Sí' if contiene_segundo_nombre else 'No'}"

# ✅ Registrar como UDF
procesar_udf = udf(procesar_nombre, StringType())

# ✅ Aplicar UDF
df_udf = df.withColumn("resultado_udf", procesar_udf(col("nombre_completo")))
df_udf.show(truncate=False)

In [0]:
# ✅ Registrar la UDF como función SQL
spark.udf.register("procesar_nombre_sql", procesar_nombre, StringType())

# ✅ Crear una vista temporal para usar en SQL
df.createOrReplaceTempView("clientes")

# ✅ Usar la UDF en una consulta SQL
df_sql = spark.sql("""
    SELECT
        id,
        nombre_completo,
        procesar_nombre_sql(nombre_completo) AS resultado_sql
    FROM clientes
""")

df_sql.show(truncate=False)

# 📌 Ejercicio – Limpieza de texto con funciones UDF

## 🔍 Objetivo:
Aplicar funciones UDF en PySpark para limpiar y normalizar textos provenientes de un archivo CSV con datos de contacto.

⸻

## 🧪 ¿Qué contiene el archivo?

El dataframe tiene 2 columnas:
- nombre_completo: incluye errores de mayúsculas, espacios extras y valores vacíos.
- email: contiene correos con formatos mixtos, nulos o mal escritos.

```python
data = [
    ['carlos   RAMIREZ', 'pedro@mail'],
    ['    ', None],
    ['carlos   RAMIREZ', 'JUAN@correo.COM'],
    ['PEDRO', 'ana_gomez@mail.com '],
    ['ANA gomez', 'JUAN@correo.COM'],
    ['PEDRO', ''],
    ['ANA gomez', ' '],
    ['  jUAN   pérez  ', ' carlos.ramirez@empresa.net'],
    ['carlos   RAMIREZ', 'ana_gomez@mail.com '],
    ['PEDRO', 'JUAN@correo.COM'],
    ['carlos   RAMIREZ', 'JUAN@correo.COM'],
    ['PEDRO', 'ana_gomez@mail.com '],
    ['carlos   RAMIREZ', 'JUAN@correo.COM'],
    ['    luisa fernanda   ', 'JUAN@correo.COM'],
    ['ANA gomez', 'ana_gomez@mail.com '],
    ['ANA gomez', 'JUAN@correo.COM'],
    ['carlos   RAMIREZ', 'JUAN@correo.COM'],
    ['  jUAN   pérez  ', 'marta.lopez@dominio.org'],
    ['MARTA   lopez rodriguez', ''],
    ['PEDRO', 'JUAN@correo.COM']
]
columns = ['nombre_completo', 'email']
df_contactos = spark.createDataFrame(data, schema=columns)
```

## 🛠 ¿Qué deben hacer?
- Leer el archivo contactos_con_errores.csv usando PySpark.
- Crear y registrar una función UDF llamada limpiar_nombre que:
  - Elimine espacios duplicados
	- Capitalice correctamente el nombre completo
	- Devuelva None si el texto está vacío o es nulo
- Crear otra UDF llamada normalizar_email que:
	- Elimine espacios en blanco
	- Pase a minúsculas
	- Detecte si tiene un formato válido (contiene @ y .)
	- Devuelva "email_invalido" si no cumple el formato
- Aplicar ambas UDFs al DataFrame original.
- Mostrar el resultado final limpio usando .show().

## funciones de referencia

```python

def limpiar_nombre(nombre):
    if not nombre or nombre.strip() == "":
        return None
    nombre_limpio = " ".join(nombre.strip().lower().split())
    return " ".join([w.capitalize() for w in nombre_limpio.split()])

def normalizar_email(correo):
    if not correo or correo.strip() == "":
        return "email_invalido"
    correo = correo.strip().lower()
    return correo if re.match(r"[^@]+@[^@]+\.[^@]+", correo) else "email_invalido"

```

📂 Entrega esperada
- Script .py o notebook .ipynb
- Aplicación de ambas UDFs
- Ejemplo con .show() de los resultados limpios


In [0]:
# escriba aqui si script